## <b>Importing Libraries:</b>

In [ ]:
!pip install -Uqq timm

In [ ]:
!pip install fastai

In [ ]:
import timm
from pathlib import Path
from fastai.vision.all import *
from fastai.vision.all import vision_learner, get_image_files

In [ ]:
path = Path("/kaggle/input/shoes-classification-dataset-13k-images/Shoes Dataset")
path.ls()

### <b>Data Exploration:</b>

In [ ]:
trn_path = path/'Train'
files = get_image_files(trn_path)

In [ ]:
img = PILImage.create(files[0])
print(img.size)
img.to_thumb(128)

- Checking the sizes of images in the dataset

In [ ]:
from fastcore.parallel import *

def f(o): return PILImage.create(o).size
sizes = parallel(f, files, n_workers=8)
pd.Series(sizes).value_counts()

### <b>Data Pre-processing:</b>

In [ ]:
trn_path = Path('/sml')

In [ ]:
resize_images(path/'Train',dest=trn_path,max_size=224,recurse=True)#resizing all the images in the folder

In [ ]:
dls = ImageDataLoaders.from_folder(trn_path, valid_pct=0.2, seed=42,item_tfms=Resize((224,168)))
dls.show_batch(max_n=3)

In [ ]:
#function to help us minimise the workflow
def train(arch, item, batch, epochs=5):
    dls = ImageDataLoaders.from_folder(trn_path, seed=42, valid_pct=0.2, item_tfms=item, batch_tfms=batch)
    metrics=[accuracy]
    learn = vision_learner(dls, arch, metrics=metrics).to_fp16() #loss function is cross entropy ,default
    learn.fine_tune(epochs, 0.01)
    return learn

- **Using Transfer Learning to classify the images in our dataset**
- **Also implement the models alongside various data augmentation techniques**
- **We will try different architechtures and choose depending on the results produced**

### <b>Model:Resnet26d</b>

In [ ]:
arch = "resnet26d"

#### <b>Pre-processing Experiments:</b>

In [ ]:
learn = vision_learner(dls,"resnet26d", metrics=[accuracy]).to_fp16()

In [ ]:
learn.lr_find() #finding the learning rate

In [ ]:
train(arch,item=Resize(224,method='squish'),batch=aug_transforms(size=224,min_scale=0.75)) #squish the images

- This gives our model an accuracy of 87 percent. 

In [ ]:
train(arch,item=Resize(224),batch=aug_transforms(size=224,min_scale=0.75)) #crop the images

- While the accuracy seems to have a slight improvement , the test time augmentation error is slightly higher than the previous one

In [ ]:
learn = train(arch, item=Resize((224,168), method=ResizeMethod.Pad, pad_mode=PadMode.Zeros),
              batch=aug_transforms(size=(168,128), min_scale=0.75)) #padding the images and resizing

- Resizing the images to a lower size does seem to have an impact on the model

### <b>Model:convnext_small_in22k</b>

In [ ]:
arch = 'convnext_small_in22k'

#### <b>Pre-processing Experiments:</b>

In [ ]:
train(arch,item=Resize(224),batch=aug_transforms(size=224,min_scale=0.75)) #cropping the images and square like images

- ConvNext seems to perform way better,gives us improved results.

In [ ]:
learn = train(arch,item=Resize((224,168),method='squish'),batch=aug_transforms(size=224,min_scale=0.75))
#squish the images and rectangle images

In [ ]:
train(arch, item=Resize((224,168), method=ResizeMethod.Pad, pad_mode=PadMode.Zeros),
              batch=aug_transforms(size=(168,128), min_scale=0.75)) #padding the images and rectangle like images

- Padding the images seems to have lesser accuracy compared to cropping the images.

### <b>Model:convnext_tiny_hnf</b>

In [ ]:
arch ="convnext_tiny_hnf"

#### <b>Pre-processing Experiments:</b>

In [ ]:
train(arch,item=Resize(224),batch=aug_transforms(size=224,min_scale=0.75))

- **From our experiments above we are able to see ConvNext models seems to perform better compared to Resnets.**
- **Cropping the images gives much better performance compared to padding or squishing the images when trying to classify**

### <b>Choosing the Final Model:</b>
- From our above experiments we can choose the **convnext_small_in22k** as the model to classify images

In [ ]:
arch = 'convnext_small_in22k'

- We will increase the size of the images leading to better Image classification.

In [ ]:
learn = train(arch,epochs=10,item=Resize((360,320), method=ResizeMethod.Pad, pad_mode=PadMode.Zeros),
              batch=aug_transforms(size=(275,275), min_scale=0.75)) 

- **Progressive resizing has improved the accuracy of our model!**

### <b>Interpreting Results of Model:</b>

In [ ]:
learn.show_results()

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix()

In [ ]:
interp.plot_top_losses(6,figsize = (25,5))

### <b>Prediction using the Trained Model:</b>
- Let us randomly input a image from the test dataset to see how it predicts

In [ ]:
learn.predict('/kaggle/input/shoes-classification-dataset-13k-images/Shoes Dataset/Test/Clog/Clog-Test (104).jpeg')

- **Gives us a accurate prediction!!**

### **Saving and Loading the Model:**

In [ ]:
learn.export(Path("/kaggle/working/export2.pkl"))
learn.model_dir = "/kaggle/working"

In [ ]:
#storing the requirements needed to run the model
!pip freeze > /kaggle/working/requirements.txt

#### **Thanks for viewing the notebook!**